In [1]:
#필요한 모듈 임포트

import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

#!pip install konlpy
#!pip install lightgbm
#!pip install catboost
#!pip install py-hanspell
from hanspell import spell_checker

from konlpy.tag import Komoran
from konlpy.tag import Okt
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Mecab

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm
from lightgbm import LGBMClassifier
import catboost
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score

In [2]:
# 데이터 수집
df = pd.read_csv('./train.csv')
df_t = pd.read_csv('./test.csv')
sub = pd.read_csv('./sample_submission.csv')

In [3]:
# 결측치 확인
df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [4]:
df_t.isnull().sum()

id          0
document    0
dtype: int64

In [120]:
# 결측값이 없으므로 결측치 처리는 생략

In [121]:
# 데이터 길이 확인
# 학습용 데이터와 제출용 테스트 데이터의 전처리 및 벡터화 과정을 동시에 진행하기 위해 병합 후
# 모델 학습 전 확인한 길이를 이용해 분리 예정
len(df)

5000

In [122]:
len(df_t)

5000

In [123]:
# 라벨 분포 확인 

In [124]:
df.label.value_counts() #1과 0의 비율이 비슷함

0    2564
1    2436
Name: label, dtype: int64

In [125]:
# 데이터 전처리 (원활한 전처리를 위해 df와 df_t의 document 열을 합친 df_m 생성, cleansing 및 맞춤법 정리 후 다시 분리)

In [5]:
df_m = df['document']
df_m = df_m.append(df_t.document, ignore_index=True)
len(df_m)

10000

In [6]:
# 정규식을 이용한 cleansing
df_m = df_m.apply(lambda x:re.sub('\d+'," ",x)) #숫자를 공백으로
df_m = df_m.apply(lambda x:re.sub('[^a-zA-Z가-힣_]+'," ",x)) #영어와 한글이 아닌 문자 및 자음만 있는 문자를 공백으로
df_m = df_m.apply(lambda x:re.sub(' +'," ",x)) #이중 공백을 공백으로

In [ ]:
# 맞춤법 정리 by hanspell
c = []
for i in range(len(df_m)):
    result = spell_checker.check(df_m[i])
    a = result.as_dict()
    b =[] 
    b.append(a['words'].keys())
    c.append(b)
documents=[]
for i in range(len(c)):
    d=[]
    for j in c[i][0]:
        d.append(j)
        e = " ".join(d)
        
    documents.append(e)

In [8]:
# 맞춤법 검사 결과를 시리즈에 저장
df_c = pd.Series(documents)

In [ ]:
# 불용어 처리

In [ ]:
# 불용어 사전 제작
# cleansing 까지 진행한 학습용 데이터(df_m이 아닌 df)를 label 값에 따라 분리 후 데이터 프레임 생성
p = df[df['label']==1]
n = df[df['label']==0]
p.reset_index(inplace=True)
n.reset_index(inplace=True)
p = p.drop(['index'], axis=1)
n = n.drop(['index'], axis=1)

In [ ]:
# 품사 추출
p_pos = [] 
p_pos_sep = [] # 단어별 빈도 데이터프레임 제작용 리스트
n_pos = []
n_pos_sep = []
for sentence in p['document']: 
    # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받음
    pos = okt.pos(sentence,stem=True) 
    
    # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] 
    
    # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
    main_words_str = " ".join(main_words) 
    p_pos.append(main_words_str)
    
    # p_pos_sep의 경우 하나의 문자열 대신 ,로 분리된 여러 문자로 저장
    p_pos_sep.append(main_words)
p["pos"] = pd.DataFrame(p_pos)

# 부정 리뷰에 대해서도 같은 작업 진행
for sentence in n['document']: 
    pos = okt.pos(sentence,stem=True)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] 
    main_words_str = " ".join(main_words)
    n_pos_sep.append(main_words)
    n_pos.append(main_words_str) 
n["pos"] = pd.DataFrame(n_pos)

In [ ]:
# 추출된 형태소와 빈도를 나타내는 빈도df 생성
p_freq = pd.Series(np.concatenate([w for w in p_pos_sep])).value_counts()
n_freq = pd.Series(np.concatenate([w for w in n_pos_sep])).value_counts()
n_freq_df = pd.DataFrame(n_freq.index, columns = ['morphs'])
p_freq_df = pd.DataFrame(p_freq.index, columns = ['morphs'])
n_freq_df['freq']=n_freq.values
p_freq_df['freq']=p_freq.values

In [ ]:
# 낮은 빈도 단어 확인
p_freq_df[p_freq_df['freq']<??] # ?? 자리에 원하는 숫자 넣기

In [ ]:
n_freq_df[n_freq_df['freq']<??]

In [ ]:
# 낮은 빈도 단어 제거
# 빈도 확인 후 해당 인덱스를 이용해 제거함
p_freq_df = p_freq_df.iloc[:??,:] # ?? 자리에 인덱스 번호 넣기
n_freq_df = n_freq_df.iloc[:??,:]

In [ ]:
# 긍정과 부정리스트 둘 다에 중복되는 단어들을 불용어로 선정 
# dup_list4 => dup_words로 저장, 빈도50미만 제거 ver.
dup_list4 = []
for i in p_freq_df['morphs'].values:
    if i in n_freq_df['morphs'].values:
        dup_list4.append(i)
for i in n_freq_df['morphs'].values:
    if i in p_freq_df['morphs'].values:
        dup_list4.append(i)
dup_list4 = list(set(dup_list4)) # 집합을 이용해 중복 단어 제거

In [ ]:
# dup_list7 => dup_words2로 저장
# 긍정, 부정 리스트 상위 20개 이내에서 (주관적으로) 유의미하게 겹치는 단어 수작업 선정
dup_list7 = ['영화', '하다', '보다', '너무', '정말', '이다', '진짜','있다']

In [ ]:
# 불용어 사전 txt파일로 저장
dup_words2 = pd.Series(dup_list7)
dup_words2.to_csv('./dup_words2.txt', index=False)

In [42]:
# 불용어 사전 생성(파일 명에 따라 3가지 ver 사전 사용)
with open('./dup_words2.txt', encoding='utf-8') as fin:
  stopwords = fin.read().split('\n')
stopwords = set(stopwords)
# 사용한 사전 리스트
# korean_stopwords
# dup_words
# dup_words2 

In [43]:
#불용어 정리 tokenizer 함수 생성
def make_token_without_stopwords(text):
  okt = Okt()
  tokens_In = okt.morphs(text, stem=True)
  tokens_Out = [word for word in tokens_In if word not in stopwords] 
  return tokens_Out

In [9]:
#불용어 정리를 하지 않는 tokenizer 함수 생성
def no_stopwords(text):
  okt = Okt()
  tokens_In = okt.morphs(text, stem=True)
  tokens_Out = [word for word in tokens_In]
  return tokens_Out

In [10]:
# 합쳤던 데이터를 다시 원래대로 분리, X,y 정의
X = df_c[:5000]
X_t = df_c[5000:]
y = df['label']

In [132]:
# 텍스트 벡터화
tfidf = TfidfVectorizer(min_df = 3, max_df = 0.9, tokenizer = no_stopwords, token_pattern = None)
X=tfidf.fit_transform(X)
X_t = tfidf.transform(X_t)
# 불용어 정리시 tokenizer = make_token_without_stopwords 사용

In [133]:
# 학습용 데이터셋과 검증용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, stratify=y,random_state=42)

In [134]:
# 분류 모델 생성
# Logistic Reg.
lr = LogisticRegression(C=2.5)
lr.fit(X_train, y_train)
train_score = lr.score(X_train, y_train)
test_score = lr.score(X_test,y_test)
print(train_score,'\n',test_score)

0.9484692331009397 
 0.8653733098177543


In [ ]:
# 불용어 사전 별 train/ test score
# korean_stopwords => 0.96/ 0.832
# dup_words => 0.93/ 0.85
# dup_words2 => 0.932/0.857
# no stopword => 0.93/0.864 <- 선정

In [ ]:
# Logistic 하이퍼 파라미터 튜닝
params = {'C':np.arange(0,5,0.5), 'penalty':['l1','l2','none','elasticnet'], 'l1_ratio':np.arange(0,1,0.1), 'max_iter':np.arange(0,1000,100)}
grid = GridSearchCV(lr, param_grid=params, scoring='accuracy',cv=10, verbose=True)
grid.fit(X_train, y_train)
best_lr = grid.best_estimator_
best_lr.fit(X_train, y_train)
train_score = best_lr.score(X_train, y_train)
test_score = best_lr.score(X_test,y_test)
print(train_score,'\n',test_score)

In [ ]:
# best hyperparameter 확인  -> C=2.5 결정
best_lr

In [11]:
# tfidf vectorizer 하이퍼파라미터 튜닝 -> max_feature = 11200 결정

X = df_c[:5000]
X_t = df_c[5000:]
y = df['label']

for max_features in range(3000, 12000, 100):
    tfidf = TfidfVectorizer(min_df=0.0, max_df=0.9, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features, tokenizer = no_stopwords, token_pattern = None)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    X_train = tfidf.fit_transform(X_train)
    X_test = tfidf.transform(X_test)
    lr = LogisticRegression(C=2.5)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    print(f"max_features: {max_features}, Accuracy: {lr.score(X_test, y_test)}" )

max_features: 3000, Accuracy: 0.854
max_features: 3100, Accuracy: 0.8566666666666667
max_features: 3200, Accuracy: 0.8586666666666667
max_features: 3300, Accuracy: 0.8573333333333333
max_features: 3400, Accuracy: 0.858
max_features: 3500, Accuracy: 0.8586666666666667
max_features: 3600, Accuracy: 0.8606666666666667
max_features: 3700, Accuracy: 0.8593333333333333
max_features: 3800, Accuracy: 0.8633333333333333
max_features: 3900, Accuracy: 0.8626666666666667
max_features: 4000, Accuracy: 0.864
max_features: 4100, Accuracy: 0.866
max_features: 4200, Accuracy: 0.8653333333333333
max_features: 4300, Accuracy: 0.8633333333333333
max_features: 4400, Accuracy: 0.8646666666666667
max_features: 4500, Accuracy: 0.8633333333333333
max_features: 4600, Accuracy: 0.862
max_features: 4700, Accuracy: 0.8626666666666667
max_features: 4800, Accuracy: 0.8626666666666667
max_features: 4900, Accuracy: 0.862
max_features: 5000, Accuracy: 0.8633333333333333
max_features: 5100, Accuracy: 0.8633333333333333


In [12]:
# tfidf vectorizer 하이퍼파라미터 튜닝 -> ngram_range=(1,3) 결정

X = df_c[:5000]
X_t = df_c[5000:]
y = df['label']

for n in range(3,30):
    tfidf = TfidfVectorizer(min_df=0.0, max_df=0.9, analyzer="char", sublinear_tf=True, ngram_range=(1,n), max_features=11200, tokenizer = no_stopwords, token_pattern = None)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    X_train = tfidf.fit_transform(X_train)
    X_test = tfidf.transform(X_test)
    lr = LogisticRegression(C=2.5)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    print(f"ngram_range_n: {n}, Accuracy: {lr.score(X_test, y_test)}" )

ngram_range_n: 3, Accuracy: 0.8713333333333333
ngram_range_n: 4, Accuracy: 0.8693333333333333
ngram_range_n: 5, Accuracy: 0.8693333333333333
ngram_range_n: 6, Accuracy: 0.8686666666666667
ngram_range_n: 7, Accuracy: 0.8686666666666667
ngram_range_n: 8, Accuracy: 0.8686666666666667
ngram_range_n: 9, Accuracy: 0.8666666666666667
ngram_range_n: 10, Accuracy: 0.87
ngram_range_n: 11, Accuracy: 0.87
ngram_range_n: 12, Accuracy: 0.8693333333333333
ngram_range_n: 13, Accuracy: 0.868
ngram_range_n: 14, Accuracy: 0.8666666666666667
ngram_range_n: 15, Accuracy: 0.8673333333333333
ngram_range_n: 16, Accuracy: 0.868
ngram_range_n: 17, Accuracy: 0.8653333333333333
ngram_range_n: 18, Accuracy: 0.866
ngram_range_n: 19, Accuracy: 0.864
ngram_range_n: 20, Accuracy: 0.8673333333333333
ngram_range_n: 21, Accuracy: 0.8673333333333333
ngram_range_n: 22, Accuracy: 0.866
ngram_range_n: 23, Accuracy: 0.866
ngram_range_n: 24, Accuracy: 0.866
ngram_range_n: 25, Accuracy: 0.8673333333333333
ngram_range_n: 26, Acc

In [17]:
# 다시 벡터화 및 분석 진행

X = df_c[:5000]
X_t = df_c[5000:]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tfidf = TfidfVectorizer(min_df=0.0, max_df=0.9, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=11200, tokenizer = no_stopwords, token_pattern = None)
X_train = tfidf.fit_transform(X_train)
X_t = tfidf.transform(X_t)
lr = LogisticRegression(C=2.5)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_t)

In [16]:
# 제출 파일 생성
sub['label']= y_pred
sub.to_csv('./sample_submission.csv', index=False)